In [1]:
import tensorflow as tf

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline

/home/minfa/.local/share/virtualenvs/cs231n-fashion-rD9JfZw-/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
buffer_size = 2
batch_size = 2
num_parallel_batches = 2

def map_fn(image, label):
    return image, label
    

def input_fn():
    d1 = tf.reshape(
        tf.range(4*3),
        (4, 3)
    )
    d2 = tf.range(4)
    dataset = tf.data.Dataset.from_tensor_slices((d1, d2))
    
    dataset = (
        dataset
        .apply(tf.contrib.data.shuffle_and_repeat(buffer_size))
        .apply(tf.contrib.data.map_and_batch(map_fn, batch_size, num_parallel_batches))
    )
    
    iterator = dataset.make_one_shot_iterator()
#     return iterator
    return iterator.get_next()

In [3]:
with tf.Session() as sess:
    next_batch = input_fn()
    
    for i in range(6):
        if i % 2 == 0:
            print('---')
        print(sess.run(next_batch))
        
    next_batch2 = input_fn()
    
    print('\n****** next_batch2 ******\n')
    for i in range(6):
        if i % 2 == 0:
            print('---')
        print(sess.run(next_batch2))

# dataset = tf.data.Dataset.from_tensor_slices((
#     tf.random_uniform([4, 10], maxval=100, dtype=tf.int32),  # train data
#     tf.random_uniform([4])  # train label
# ))

Instructions for updating:
Use the retry module or similar alternatives.
---
(array([[3, 4, 5],
       [0, 1, 2]], dtype=int32), array([1, 0], dtype=int32))
(array([[ 9, 10, 11],
       [ 6,  7,  8]], dtype=int32), array([3, 2], dtype=int32))
---
(array([[0, 1, 2],
       [3, 4, 5]], dtype=int32), array([0, 1], dtype=int32))
(array([[ 9, 10, 11],
       [ 6,  7,  8]], dtype=int32), array([3, 2], dtype=int32))
---
(array([[0, 1, 2],
       [6, 7, 8]], dtype=int32), array([0, 2], dtype=int32))
(array([[ 3,  4,  5],
       [ 9, 10, 11]], dtype=int32), array([1, 3], dtype=int32))

****** next_batch2 ******

---
(array([[0, 1, 2],
       [3, 4, 5]], dtype=int32), array([0, 1], dtype=int32))
(array([[ 9, 10, 11],
       [ 6,  7,  8]], dtype=int32), array([3, 2], dtype=int32))
---
(array([[3, 4, 5],
       [6, 7, 8]], dtype=int32), array([1, 2], dtype=int32))
(array([[ 9, 10, 11],
       [ 0,  1,  2]], dtype=int32), array([3, 0], dtype=int32))
---
(array([[0, 1, 2],
       [6, 7, 8]], dtype=i

In [ ]:
def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((
        tf.random_uniform([4, 10], maxval=100, dtype=tf.int32),  # train data
        tf.random_uniform([4])  # train label
    ))
    
    
    
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [5]:
def model_fn(features, labels, mode, params):
    return tf.estimator.EstimatorSpec(mode)

In [6]:
estimator = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpsxbk1xuu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe7f7256240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [2]:
estimator.train(input_fn, steps=1)

In [3]:
import os

In [8]:
print(os.environ.get('hello_world_env'))

None


In [1]:
import logging

In [ ]:
logging.FileHandler()

# Model Ensemble

In [3]:
import os
import importlib
import tqdm

MODEL_DIR_PREFIX = '/home/shared/cs231n-fashion/model_dir'

def write_predictions(probs):
    """
    Inputs:
        probs: 
    
    Globals:
        FLAGS.test_prediction
        FLAGS.pred_threshold
    """
    print("Saving test data to: ", FLAGS.test_prediction)
    f = open(FLAGS.test_prediction, "w")
    f.write("image_id,label_id\n")
    img_id = 1

    # deal with unified threshold or per class thresholding
    thresholds = []
    if re.match("^\d+?\.\d+?$", FLAGS.pred_threshold) is None:
        print("Use per class thresholding.")
        thresholds = pd.read_csv(FLAGS.pred_threshold)['thresholds'].values
    else:
        th = float(FLAGS.pred_threshold)
        thresholds = [th for i in range(228)]

    with tqdm(total=NUM_TEST) as progress_bar:
        for prob in probs:
            labels=" ".join([
                str(i+1)
                for i in range(len(prob))
                if prob[i] >= thresholds[i]
            ])
            f.write("%d,%s\n"%(img_id, labels))
            img_id += 1
            progress_bar.update(1)
    print("Processed %d examples. Good Luck! :)"%(img_id))
    f.close()
    

def predict_with_ensemble(ensemble_label_to_model_meta, run_config, params, test_input_fn):
    """
    Inputs:
        ensemble_label_to_model_meta: dict. KV is in the following format:
            'label': ('model_file', 'exp_name', weight).
            The model_file should export model_fn(features, labels, mode, params).
            The exp_name should be the folder name that stores the weights of the model.
    """
    # Validate existence of exp_names.

    total_weight = 0.0
    agg_probs = None

    for label in ensemble_label_to_model_meta:
        model_file, exp_name, weight = ensemble_label_to_model_meta[label]
        model_dir = os.path.join(MODEL_DIR_PREFIX, exp_name)

        # Create model from model_file.
        model_fn = importlib.import_module(model_file).model_fn

        # Initialize classifier with weights from exp_name.
        classifier = tf.estimator.Estimator(
            config=run_config,
            model_fn=model_fn,
            model_dir=model_dir,
            params=params
        )

        # Make predictions.
        probs = [pred['probs'] for pred in classifier.predict(test_input_fn)]
        
        # Aggregate weighted probs.
        if agg_probs is None:
            agg_probs = weight * probs
        else:
            agg_probs += weight * probs
        total_weight += weight
    
    agg_probs /= total_weight
    write_predictions(agg_probs)

In [ ]:
# KV is 'label': ('model_file', 'exp_name', weight)
ensemble_label_to_model_meta = {
    'label1': ['base', 'exp1', 0.2],
    'label2': ['base', 'exp2', 0.5],
}

run_config = tf.estimator.RunConfig(
    session_config=tf.ConfigProto(log_device_placement=True),
#             save_checkpoints_secs=30*60,
#             keep_checkpoint_max=10,
)

params = {
    'learning_rate': FLAGS.learning_rate,
    'num_classes': FLAGS.num_classes,
    'module_trainable': FLAGS.module_trainable,
    'eval_thresholds': [float(i) for i in FLAGS.eval_thresholds.split(';')],
    'model_dir': FLAGS.model_dir,
    'reg': FLAGS.reg,
}

In [2]:
import logging

In [4]:
log = logging.getLogger('tensorflow')

In [9]:
stdout_handler = log.handlers[0]

In [10]:
log.removeHandler(stdout_handler)

# Test ensemble

1. Read csv file using pandas
2. 

In [11]:
import pandas as pd

In [19]:
import numpy as np
import pandas as pd
import os

prob_dir = '/home/minfa/ensemble_dir'
prob_files = os.listdir(prob_dir)
agg_probs = None

for prob_file in prob_files:
    prob_file_path = os.path.join(prob_dir, prob_file)
    print("Reading data from: {}".format(prob_file_path))
    
    probs = pd.read_csv(prob_file_path)['label_prob'].map(lambda x: np.array([float(v) for v in x.split(' ')])).values
    probs = np.array(probs.tolist())
    if agg_probs is None:
        agg_probs = probs
    else:
        agg_probs += probs

agg_probs /= len(prob_files)

Reading data from: /home/minfa/ensemble_dir/debug_dump_test.csv
Reading data from: /home/minfa/ensemble_dir/debug_dump_test2.csv
